# Estimating nuisance parameters in inverse problems

Based on 

* [Estimating nuisance parameters in inverse problems](https://iopscience.iop.org/article/10.1088/0266-5611/28/11/115016).
A.Y. Aravkin and T. van Leeuwen. Inverse Problems (28:11)

* [Variable Projection for NonSmooth Problems](https://epubs.siam.org/doi/abs/10.1137/20M1348650). T. van Leeuwen and A.Y. Aravkin. SIAM J. Sci. Comput. (accepted)

(Motivation)

# Overview

* General formulation
* Variable projection
* Examples
* Wrap-up

# General formulation

MAP estimation for inverse problems can be generally formulated as 

$$\min_u F(u) + G(u),$$

with $F(u) = -\log \pi_{\text{data}}(u)$ and  $G(u) = -\log \pi_{\text{prior}}(u)$.

The densities often include parameters $\theta$ (mean, variance, ...). This leads to 

$$\min_{u,\theta} F(u,\theta) + G(u,\theta) + H(\theta),$$

with $H(\theta) = -\log \pi_{\text{hyper}}(\theta)$.

> **Example: *Extended least-squares:***
> $$\min_{u,\Sigma} m \log |\Sigma| + \sum_{i=1}^m \|K(u) - f^\delta\|_{\Sigma^{-1}}^2.$$


> **Example: *Robust data-fitting:***
> $$\min_{u,\sigma^2} m\log\sigma + \sum_{i=1}^m \log\left(1 + \frac{r_i(u)^2}{\sigma^2}\right), \quad r(u) = K(u) - f.$$

> **Example: *sparse regularisation:***

Eliminate $u$ and formulate

$$\min_{\theta} \left(\min_u F(u,\theta) + G(u,\theta) + H(\theta)\right).$$

* Evaluation for given $\theta$ requires solution of inverse problem
* Results in non-linear problem in $\theta$ with expensive function evaluations
* Can possibly use surrogate modelling

# Variable projection


Often, the nuisance parameters separate, and we can write

$$\min_u \left(\underbrace{\min_\alpha F(u,\alpha)}_{\overline{F}(u)} + \underbrace{\min_\beta G(u,\beta)}_{\overline{G}(u)}\right).$$


and apply a proximal-gradient algorithm

$$u_{k+1} = \text{prox}_{\lambda \overline{G}} \left(u_k - \lambda \nabla \overline{F}(u)\right).$$


## Derivative formula
Assuming that $\nabla_u F$ and $\nabla_\alpha F$ are Lipschitz-continuous and $F$ is $\mu-$ strongly convex in $\alpha$ for all $u$;


* $\overline{\alpha}(u) = \text{arg}\min_\alpha F(u,\alpha)$ is Lipschitz continuous 
* $\nabla \overline{F}(u) = \left.\nabla_u F(u,\alpha)\right|_{\alpha = \overline{\alpha}(u)}.$
* An approximate minimiser $\widetilde{\alpha}(u)$ yields an approximate gradient with

$$\|\nabla F(u,\overline{\alpha}(u)) - \nabla F(u,\widetilde{\alpha}(u))\| \leq L_{u\alpha} \|\overline{\alpha}(u) -\widetilde{\alpha}(u)\|.$$

## Proximal operator

$$\text{prox}_{\lambda \overline{G}}\left(v\right) = \text{arg}\min_{u,\beta} \textstyle{\frac{1}{2}}\|u - v\|_2^2 + G(u,\beta).$$

* Still a proximal operator as long as $\overline{G}$ is convex
* Solution may be available in closed-form
* Approximate solution is ok, as long as $\min_u \textstyle{\frac{1}{2}}\|u - v\|_2^2 + G(u,\widetilde{\beta}(u)) \leq \delta + \min_{u} \textstyle{\frac{1}{2}}\|u - v\|_2^2 + \overline{G}(u).$

## Convergence results

Convergence of proximal gradient with inexact gradients ($\epsilon_k$) and inexact proximal operators ($\delta_k$)[^1]:

* sublinear ($\mathcal{O}(1/k)$) convergence rate for *convex* problems if $\{\epsilon_k\}$ and $\{\delta_k\}$ are summable
* linear convergenge if $\{\epsilon_k\}$ and $\{\delta_k\}$ decrease linearly

[^1]:Schmidt, Mark, Nicolas Le Roux, and Francis Bach. "Convergence rates of inexact proximal-gradient methods for convex optimization." NeurIps (2011).

## Implementation

* Closed-form solution may be available
* Can be solved to desired accuracy when $\theta$ represents only a few variables
* Able to re-use existing code for gradient computations and optimisation


# Examples

## Tikhonov regularisation

Given $K \in \mathbb{R}^{m\times n}$ and $f^\delta = Ku_{\text{true}} + \epsilon$ with $\epsilon \sim N(0,\sigma^2)$, solve

$$\min_{u,\alpha,\beta} \alpha \|Ku - f^\delta\|_2^2 + \beta \|Lu\|_2^2 - m \log \alpha - n\log\beta.$$

* $\overline{\alpha}(u) = m\|Ku - f^\delta\|_2^{-2}$, $\overline{\beta}(u) = n\|Lu\|_2^{-2}$ (solve for $u$ with BFGS)
* $\overline{u}(\alpha,\beta) = \left(K^*\!K + (\beta/\alpha)L^*\!L\right)^{-1}K^*f^\delta$ (solve for $(\alpha,\beta)$ with Nelder-Mead)


![](./figures/Tikh_u_hat.png)

![](./figures/Tikh_theta_hat.png)

![](figures/Tikh_convergence.png)

## Covariance estimation

Non-linear seismic inversion of multi-experiment data.

* $F(u,\Sigma) = m\log|\Sigma| + \sum_{i=1}^m\|K(u)q_i - f_i^\delta\|_{\Sigma^{-1}}^2.$
* Use sample-covariance at each iteration
* Can use further tricks to get low-rank approximation of $\Sigma$

![](./figures/experiment5_1.png)

![](./figures/experiment4_2.png)

## Robust fitting^[1]

Tomograpraphic image reconstruction from data with outliers:

* $F(u,\alpha) = \sum_{i=1}^m \log\left(1 + \alpha(Ku-f^\delta)_i^2\right) -2m\log(\alpha).$
* $G(u) = \text{TV}(u)$
* Solve for $\alpha$ using scalar minimisation method

[^1]:Kazantsev, D., Bleichrodt, F., Van Leeuwen, T., Kaestner, A., Withers, P., Batenburg, K. J., & Lee, P. (2017). A novel tomographic reconstruction method based on the robust Student’s t function for suppressing data outliers. IEEE Transactions on Computational Imaging, 1–1. https://doi.org/10.1109/TCI.2017.2694607

![](./figures/studentst1.png)

# Wrap-up

* Algorithmic framework for estimating hyper parameters in inverse problems
* ...

Other relevant issues:

* Marginalisation of $\theta$
* Choice of Hyper-prior
* Regularising properties and convergence rates (as noise vanishes)
